# Figure 2) Spaghetti plot: 0 T (Run constant-E slices before MATLAB code)

## Import libraries and define functions

In [ ]:
import matplotlib.pyplot as plt
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass

from MJOLNIR import _tools
from MJOLNIR.Data import DataSet
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy

In [ ]:
def Gaussian(x,A,mu,sigma,B):
    return A*np.exp(-np.power(x-mu,2.0)/(2.0*sigma**2))+B
def gaussian2(x, bg, amp1, cen1, wid1, amp2, cen2, wid2):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))+amp2/(np.sqrt(2*np.pi)*wid2) * np.exp(-(x-cen2)**2 /(2*wid2**2))

In [ ]:
def cleaning(ds,treshold):
    i=0
    for dataset in range(len(ds)):
        for a3 in range(0,len(ds.a3[dataset])-1):
            if (np.sum(ds[dataset].I[a3,:,:],axis=(0,1))<treshold):
                ds[dataset].I[a3,:,:]=0
                ds[dataset].Monitor[a3,:,:]=0
                i=i+1
    return i

## Read in data

### (HHL)

In [ ]:
files = _tools.fileListGenerator("2588-2608",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHHL = DataSet.DataSet(files)
dsHHL.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHHL,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsHHL):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

### (H0L)

In [ ]:
files = _tools.fileListGenerator("1371-1388",r"U:\Data\CAMEA\MnF2\hdfH0L",2021)
dsH0L = DataSet.DataSet(files)
dsH0L.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsH0L,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsH0L):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

### (HK0)

In [ ]:
files = _tools.fileListGenerator("2745-2767",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHK0 = DataSet.DataSet(files)
dsHK0.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHK0,treshold)
print(i)

# Symmetrize
for i,df in enumerate(dsHK0):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

## QE slice along high-symmetry paths

### (HHL)

In [ ]:
grid = False
dataFiles = None
rlu = True
dss=dsHHL
Q1 = np.array([0,0,1]) # Gamma
Q2 = np.array([0.5,0.5,1])  # M
Q3 = np.array([0.5,0.5,0.5]) # A
Q4 = np.array([1,1,0.5]) # Z
Q5 = np.array([1,1,0]) # Gamma
EMin = 0  #Energy minimum
EMax = 8.45 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.04 # integration width perpendicular to Q in 1/A
minPixel = 0.02 # step along Q in 1/A

vmin = 0.0 # caxis minimum
vmax = 2 # caxis maximum
axFG1,data = dss.plotCutQELine(QPoints=np.array([Q1,Q2,Q3,Q4,Q5]),EnergyBins=energy,width=width,minPixel=minPixel,rlu=rlu,plotSeperator=True,seperatorWidth=1,seperatorColor='w',cmap='turbo')
#axFG1.set_title('1.6 K, 0 T')
#axFG1.locator_params(axis='x', nbins=7)
axFG1.set_ylabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)',fontsize=20)
axFG1.set_xlabel("")
axFG1.tick_params(axis='both', which='major', labelsize=20)
axFG1.set_clim(vmin, vmax)
axFG1.get_figure().set_size_inches(10,5)
#axFG1.set_xticks([0.0, 0.92, 1.88, 2.8, 3.75], (r"$\Gamma$", "M", "A", "Z", r"$\Gamma$"))
axFG1.get_xaxis().set_ticks([]);
plt.savefig('U:/Data/Figures/MnF2/Manuscript/Fig2/HHLData.png')

### (H0L)

In [ ]:
grid = False
dataFiles = None
rlu = True
dss=dsH0L
Q1 = np.array([1,0,0]) # Gamma
Q2 = np.array([1.5,0,0])  # X
Q3 = np.array([1.5,0,0.5]) # R
Q4 = np.array([1,0,0.5]) # Z
EMin = 0  #Energy minimum
EMax = 8.45 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.04 # integration width perpendicular to Q in 1/A
minPixel = 0.03 # step along Q in 1/A

vmin = 0.0 # caxis minimum
vmax = 0.1 # caxis maximum
axFG2,data = dss.plotCutQELine(QPoints=np.array([Q1,Q2,Q3,Q4]),EnergyBins=energy,width=width,minPixel=minPixel,rlu=rlu,plotSeperator=True,seperatorWidth=1,seperatorColor='w',cmap='turbo')
#axFG2.set_title('1.6 K, 0 T')
#axFG2.locator_params(axis='x', nbins=7)
#axFG2.set_ylabel(r'$\Delta \it{E}$ (meV)',fontsize=20)
axFG2.set_ylabel("")
axFG2.set_xlabel("")
axFG2.tick_params(axis='both', which='major', labelsize=20)
axFG2.set_clim(vmin, vmax)
axFG2.get_figure().set_size_inches(7.5,5)
#axFG2.set_xticks([-0.02, 0.645, 1.605, 2.25], (r"$\Gamma$", "X", "R", "Z"))
axFG2.get_xaxis().set_ticks([]);
axFG2.set_xlim(-0.02, 2.25)
plt.savefig('U:/Data/Figures/MnF2/Manuscript/Fig2/H0LData.png')

### (HK0)

In [ ]:
grid = False
dataFiles = None
rlu = True
dss=dsHK0
Q1 = np.array([1,0.5,0]) # X
Q2 = np.array([1.5,0.5,0]) # M
EMin = 0  #Energy minimum
EMax = 8.45 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.04 # integration width perpendicular to Q in 1/A
minPixel = 0.02 # step along Q in 1/A

vmin = 0.0 # caxis minimum
vmax = 1 # caxis maximum
axFG3,data = dss.plotCutQELine(QPoints=np.array([Q1,Q2]),EnergyBins=energy,width=width,minPixel=minPixel,rlu=rlu,plotSeperator=True,seperatorWidth=1,seperatorColor='w',cmap='turbo')
#axFG3.set_title('1.6 K, 0 T')
#axFG3.locator_params(axis='x', nbins=7)
#axFG3.set_ylabel(r'$\Delta \it{E}$ (meV)',fontsize=20)
axFG3.set_ylabel("")
axFG3.set_xlabel("")
axFG3.tick_params(axis='both', which='major', labelsize=20)
axFG3.set_clim(vmin, vmax)
axFG3.get_figure().set_size_inches(2.5,5)
#axFG3.set_xticks([-0.015, 0.645], ("X", "M"))
axFG3.get_xaxis().set_ticks([]);
plt.savefig('U:/Data/Figures/MnF2/Manuscript/Fig2/HK0Data.png', bbox_inches='tight')

## Constant-E Slices

In [ ]:
EMin = 3.95 #Energy minimum
EMax = 4.05 #Energy maximum
xBinTolerance = .03 #value in 1/A. binning along x
yBinTolerance = .03 #value in 1/A. binning along y
vmin=0.0 #minimum of the color scale
vmax=.7 #maximum of the color scale
Data,ax = dsHK0.plotQPlane(EMin=EMin, EMax=EMax, xBinTolerance=xBinTolerance, yBinTolerance=yBinTolerance, log=False, cmap = 'turbo')
fig = ax.get_figure() # Extract figure from returned axis
fig.colorbar(ax.pmeshs[0]) # Create colorbar from plot
ax.set_clim(vmin,vmax)
ax.set_xlabel("(001) (rlu)")
ax.set_ylabel("(110) (rlu)")
ax.tick_params(axis='both', which='major', labelsize=16)
ax.get_figure().set_size_inches(5,5)
ax.set_xlim(0, 2.01)
ax.set_ylim(0, 2.01)
ax.grid(False)

# Save Pandas DataFrame
dir = 'U:/Data/Figures/MnF2/Manuscript/Fig2/HK0'+str(round(np.mean((EMin, EMax))))+'meV.csv'
Data.to_csv(dir)

# Figure 3) High-resolution slices, cuts: 0 T (Run after MATLAB code)

## Import libraries and define functions

In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass

from MJOLNIR import _tools
from MJOLNIR.Data import DataSet
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy

In [2]:
def gaussian(x,bg, amp1, cen1, wid1):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))

In [3]:
def cleaning(ds,treshold):
    i=0
    for dataset in range(len(ds)):
        for a3 in range(0,len(ds.a3[dataset])-1):
            if (np.sum(ds[dataset].I[a3,:,:],axis=(0,1))<treshold):
                ds[dataset].I[a3,:,:]=0
                ds[dataset].Monitor[a3,:,:]=0
                i=i+1
    return i

## Read in data

### 1.5 K, 0 T, High E-resolution scan, (HHL)

In [4]:
files = _tools.fileListGenerator("2644-2659",r"U:\Data\CAMEA\MnF2\hdf",2024)
dsHR = DataSet.DataSet(files)
dsHR.convertDataFile(binning = 8,saveFile=False) 

# Mask tube 23 which is displaced in the older normalization
mask=[]
mask.append(Mask.indexMask(23,24,axis=1))
dsHR.mask = [np.logical_or(m1,m2) for m1,m2 in zip(dsHR.mask,np.sum(mask)(dsHR))]

#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(dsHR,treshold)
print(i)

0


## Plot slice from Z to A

In [5]:
%matplotlib qt
dset=dsHR
rlu = True
#grid = -10
grid = False
self = dset
dataFiles = None
rlu = True
Q1 = np.array([0.45,0.45,0.5]) #Q1 of cut
Q2 = np.array([1.05,1.05,0.5]) #Q2 of cut
EMin = 6.45 #Energy minimum
EMax = 7.35 #Energy maximum
dE = 0.05 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.03 #integration width perpendicular to Q in 1/A
minPixel = 0.03 #step along Q in 1/A
vmin=0 #caxis minimum
vmax=1 #caxis maximum

axSUB,data,bins = dset.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=True,cmap='turbo')
axSUB.set_ylabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=16)
axSUB.set_xlabel("")
axSUB.tick_params(axis='both', which='major', labelsize=16)
axSUB.set_clim(vmin, vmax)
axSUB.set_xlim(0, 1)
axSUB.get_figure().set_size_inches(10,5)
axSUB.set_xticks([0.0833, 0.917], ("A", "Z"))
plt.axvline(x=0.0833, color='m', linewidth=3.0, linestyle='--')
plt.axvline(x=0.5, color='m', linewidth=3.0, linestyle='--')
plt.axvline(x=0.917, color='m', linewidth=3.0, linestyle='--')
#cb = axSUB.get_figure().colorbar(cm.ScalarMappable(norm=None, cmap='turbo'), ax=axSUB, ticks=[0, 1])
#cb.set_label(label='$I$ (arb. units)', size=18)
#cb.ax.tick_params(labelsize=18)
cb = axSUB.colorbar
cb.set_label(label='$I$ (arb. units)', size=16)
cb.ax.tick_params(labelsize=16)
cb.set_ticks([0, 1])
plt.show()

### Overplot the dispersion

In [6]:
# Read in the fitted modes from SpinW. See https://mjolnir.readthedocs.io/en/latest/notebooks/OverviewofCAMEAData.html#The-MnF2-dispersion-theory, https://mjolnir.readthedocs.io/en/latest/notebooks/Fittingofsequential1Dcuts.html.
hklFit = np.loadtxt('U:/Data/MATLAB/MnF2/hklFit.txt')
omegaFit = np.loadtxt('U:/Data/MATLAB/MnF2/omegaFit.txt')
hklFit = np.transpose(hklFit)
XPosition = np.concatenate([axSUB.calculatePositionInv(hkl.reshape(1,-1)) for hkl in hklFit])
axSUB.plot(XPosition, omegaFit[0], color='k', linewidth=2, linestyle='-')
axSUB.plot(XPosition, omegaFit[1], color='k', linewidth=2, linestyle='-')

# Read in the split modes from SpinW. See https://mjolnir.readthedocs.io/en/latest/notebooks/OverviewofCAMEAData.html#The-MnF2-dispersion-theory, https://mjolnir.readthedocs.io/en/latest/notebooks/Fittingofsequential1Dcuts.html.
hklSplit = np.loadtxt('U:/Data/MATLAB/MnF2/hklSplit.txt')
omegaSplit = np.loadtxt('U:/Data/MATLAB/MnF2/omegaSplit.txt')
hklSplit = np.transpose(hklSplit)
XPosition = np.concatenate([axSUB.calculatePositionInv(hkl.reshape(1,-1)) for hkl in hklSplit])
axSUB.plot(XPosition, omegaSplit[0], color='w', linewidth=2, linestyle='-')
axSUB.plot(XPosition, omegaSplit[1], color='w', linewidth=2, linestyle='-')
plt.savefig('U:/Data/Figures/MnF2/Manuscript/Fig3/AZSlice.png')

## Plot E cuts

In [7]:
dss=dsHR
Q1 = np.array([0.5,0.5,0.5])  #Q-position
Q2 = np.array([0.75,0.75,0.5])  #Q-position
Q3 = np.array([1,1,0.5])  #Q-position

# A
width = 0.025 #integration width perpendicular to Q in 1/A
minPixel = 0.05 #step along Energy
EMin=6.4 #Minimum of energy
EMax=7.3 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
data, bin = dss.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
fig, (axs1, axs2, axs3)=plt.subplots(1, 3, figsize=(10,6), sharey=True)
plt.subplots_adjust(wspace = 0.03)
axs1.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=7, wid1=0.1, weights=1/data['Int_err'])
axs1.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
axs1.set_title("")
axs1.set_xlabel(r"$\mathrm{\Delta}\mathit{E}$ (meV)", fontsize=20)
axs1.set_ylabel(r"$\it{I}$ (arb. units)", fontsize=20)
#print(result.chisqr)
#axs1.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 4, linestyle = '-', color = 'k')
#axs1.legend(frameon=True)
#axs1.text(0.03, 0.95, "A" "\n" "0.190(9) meV", transform=axs1.transAxes, fontsize=20, verticalalignment='top')
axs1.text(0.7, 0.98, "0.197(6) meV", transform=axs1.transAxes, fontsize=20, verticalalignment='top', horizontalalignment='right')
axs1.set_ylim(0, 1.2)
axs1.set_xlim(6.4, 7.3)
axs1.tick_params(right = True)
axs1.tick_params(axis='both', which='major', labelsize=18, color='m', labelcolor='k')
for spine in axs1.spines.values():
    spine.set_edgecolor('m')
print("Q: " + str(Q1) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")


# Midpoint
width = 0.025 #integration width perpendicular to Q in 1/A
minPixel = 0.05 #step along Energy
EMin=6.4 #Minimum of energy
EMax=7.3 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
data, bin = dss.cut1DE(E1=EMin,E2=EMax,q=Q2,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
axs2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.9, wid1=0.1, weights=1/data['Int_err'])
axs2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
axs2.set_title("")
axs2.set_xlabel(r"$\mathrm{\Delta}\mathit{E}$ (meV)", fontsize=20)
axs2.set_ylabel("")
axs2.tick_params(labelleft = False)
#print(result.chisqr)
#axs2.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 4, linestyle = '-', color = 'k')
#axs2.legend(frameon=True)
#axs2.text(0.03, 0.95, r"$\left( \frac{3}{4} \frac{3}{4} \frac{1}{2} \right)$" "\n" "0.21(1) meV", transform=axs2.transAxes, fontsize=20, verticalalignment='top')
axs2.text(0.7, 0.98, "0.216(8) meV", transform=axs2.transAxes, fontsize=20, verticalalignment='top', horizontalalignment='right')
axs2.set_xlim(6.4, 7.3)
axs2.tick_params(right = True)
axs2.tick_params(axis='both', which='major', labelsize=18, color='m', labelcolor='k')
for spine in axs2.spines.values():
    spine.set_edgecolor('m')
print("Q: " + str(Q2) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")


# Z
width = 0.025 #integration width perpendicular to Q in 1/A
minPixel = 0.05 #step along Energy
EMin=6.4 #Minimum of energy
EMax=7.3 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
data, bin = dss.cut1DE(E1=EMin,E2=EMax,q=Q3,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
data = data.drop(data[data['Int']==0].index).reset_index(drop=True) # Drop data pandas dataframe rows with 0 counts
axs3.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)
gmodel = Model(gaussian)
result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=6.9, wid1=0.1, weights=1/data['Int_err'])
axs3.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
gaussian(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
result.params['amp1'],result.params['cen1'],result.params['wid1']),linestyle='--',color='m',linewidth=2)
axs3.set_title("")
axs3.set_xlabel(r"$\mathrm{\Delta}\mathit{E}$ (meV)", fontsize=20)
axs3.set_ylabel("")
axs3.tick_params(labelleft = False)
#print(result.chisqr)
#axs3.hlines(y = result.params['bg'].value + result.params['amp1'].value/(np.sqrt(2*np.pi)*result.params['wid1'].value)/2, xmin = result.params['cen1'].value - result.params['wid1'].value*np.sqrt(2*np.log(2)), xmax = result.params['cen1'].value + result.params['wid1'].value*np.sqrt(2*np.log(2)), linewidth = 4, linestyle = '-', color = 'k')
#axs3.legend(frameon=True)
axs3.tick_params(axis='both', which='major', labelsize=18, color='m', labelcolor='k')
for spine in axs3.spines.values():
    spine.set_edgecolor('m')
#axs3.text(0.03, 0.95, "Z" "\n" "0.21(1) meV", transform=axs3.transAxes, fontsize=20, verticalalignment='top')
axs3.text(0.7, 0.98, "0.202(9) meV", transform=axs3.transAxes, fontsize=20, verticalalignment='top', horizontalalignment='right')
axs3.set_xlim(6.4, 7.3)
print("Q: " + str(Q3) + "\n" "FWHM: " + str(round(result.params['wid1'].value*2*np.sqrt(2*np.log(2)), 3)) + r"+-" + str(round(result.params['wid1'].stderr*2*np.sqrt(2*np.log(2)), 3)) + " meV")
fig.savefig('U:/Data/Figures/MnF2/Manuscript/Fig3/ECuts.png', bbox_inches='tight')

Q: [0.5 0.5 0.5]
FWHM: 0.197+-0.006 meV
Q: [0.75 0.75 0.5 ]
FWHM: 0.216+-0.008 meV
Q: [1.  1.  0.5]
FWHM: 0.202+-0.009 meV


# Figure 4) Spaghetti plot: 10 T (Run after MATLAB code)

## Import libraries and define functions

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass

from MJOLNIR import _tools
from MJOLNIR.Data import DataSet
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy

In [ ]:
def Gaussian(x,A,mu,sigma,B):
    return A*np.exp(-np.power(x-mu,2.0)/(2.0*sigma**2))+B
def gaussian2(x, bg, amp1, cen1, wid1, amp2, cen2, wid2):
    return bg+amp1/(np.sqrt(2*np.pi)*wid1) * np.exp(-(x-cen1)**2 /(2*wid1**2))+amp2/(np.sqrt(2*np.pi)*wid2) * np.exp(-(x-cen2)**2 /(2*wid2**2))

In [ ]:
def cleaning(ds,treshold):
    i=0
    for dataset in range(len(ds)):
        for a3 in range(0,len(ds.a3[dataset])-1):
            if (np.sum(ds[dataset].I[a3,:,:],axis=(0,1))<treshold):
                ds[dataset].I[a3,:,:]=0
                ds[dataset].Monitor[a3,:,:]=0
                i=i+1
    return i

## Read in data

In [ ]:
files = _tools.fileListGenerator("2671-2694",r"U:\Data\CAMEA\MnF2\hdf",2024)
ds10T = DataSet.DataSet(files)
ds10T.convertDataFile(binning = 8,saveFile=False) 
#check if problems are in some dataset and correct for it:
treshold=50
i=cleaning(ds10T,treshold)
print(i)

# Symmetrize
for i,df in enumerate(ds10T):
    H,K,L = df.h,df.k,df.l
    df.qx,df.qy = df.sample.calculateHKLToQxQy(np.abs(H),np.abs(K),np.abs(L))

## QE slice along high-symmetry paths

In [ ]:
grid = False
dataFiles = None
rlu = True
dss=ds10T
Q1 = [0,0,1] # Gamma
Q2 = [0.5,0.5,1] # M
Q3 = [0.5,0.5,0.5] # A
Q4 = [1,1,0.5] # Z
Q5 = [1,1,0] # Gamma
EMin = 0  #Energy minimum
EMax = 8.45 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.04 # integration width perpendicular to Q in 1/A
minPixel = 0.02 # step along Q in 1/A

vmin = 0.0 # caxis minimum
vmax = 2 # caxis maximum
ax1,data = dss.plotCutQELine(QPoints=[Q1,Q2,Q3,Q4,Q5],EnergyBins=energy,width=width,minPixel=minPixel,rlu=rlu,plotSeperator=True,seperatorWidth=1,seperatorColor='w',cmap='turbo')
ax1.set_ylabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax1.set_xlabel("")
ax1.tick_params(axis='both', which='major', labelsize=18)
ax1.set_clim(vmin, vmax)
ax1.set_xlim(-0.02, 3.76);
ax1.set_xticks([-0.02, 0.915, 1.873, 2.802, 3.76], (r"$\Gamma$", "M", "A", "Z", r"$\Gamma$"));
ax1.get_figure().set_size_inches(10,5)
cb = ax1.get_figure().colorbar(cm.ScalarMappable(norm=None, cmap='turbo'), ax=ax1, ticks=[0, 1])
cb.set_label(label='$I$ (arb. units)', size=18)
cb.ax.tick_params(labelsize=18)
plt.vlines(x=0.915/4, ymin=0.26, ymax=3.85, colors='w', linewidth=2.0, linestyle='dashed')
plt.hlines(y=3.85, xmin=0, xmax=0.915/4, colors='w', linewidth=2.0, linestyle='dashed')
plt.hlines(y=0.26, xmin=0, xmax=0.915/4, colors='w', linewidth=2.0, linestyle='dashed')

### Read in the split modes from SpinW.

In [ ]:
#See https://mjolnir.readthedocs.io/en/latest/notebooks/OverviewofCAMEAData.html#The-MnF2-dispersion-theory, https://mjolnir.readthedocs.io/en/latest/notebooks/Fittingofsequential1Dcuts.html.
hklSplit = np.loadtxt('U:/Data/MATLAB/MnF2/hklFit10T.txt')
omegaSplit = np.loadtxt('U:/Data/MATLAB/MnF2/omegaFit10T.txt')
hklSplit = np.transpose(hklSplit)
# Which slice for which Q indices
hklSplit1 = hklSplit[0:223];
hklSplit2 = hklSplit[223:445];
hklSplit3 = hklSplit[445:667];
hklSplit4 = hklSplit[667:len(hklSplit)];
hklSplit1 = np.insert(hklSplit1, 3, 0, axis=1)
hklSplit2 = np.insert(hklSplit2, 3, 1, axis=1)
hklSplit3 = np.insert(hklSplit3, 3, 2, axis=1)
hklSplit4 = np.insert(hklSplit4, 3, 3, axis=1)
hklforPosition = np.concatenate((hklSplit1, hklSplit2, hklSplit3, hklSplit4))
XPosition = [ax1.calculatePositionInv(hkl.reshape(1,-1)) for hkl in hklforPosition]
ax1.plot(XPosition, omegaSplit[0], color='k', linewidth=2, linestyle='--')
ax1.plot(XPosition, omegaSplit[1], color='k', linewidth=2, linestyle='--')
plt.savefig('U:/Data/Figures/MnF2/Manuscript/Fig4/HHLData.png')

### Zoom in on the splitting

In [ ]:
grid = False
dataFiles = None
rlu = True
dss=ds10T
Q1 = [0,0,1] # Gamma
Q2 = [1/8,1/8,1]
EMin = 0  #Energy minimum
EMax = 4 #Energy maximum
dE = 0.1 #Energy steps
energy = np.arange(EMin,EMax,dE)
width = 0.04 # integration width perpendicular to Q in 1/A
minPixel = 0.02 # step along Q in 1/A

vmin = 0.0 # caxis minimum
vmax = 2 # caxis maximum
axIn,data,bins = dss.plotCutQE(Q1,Q2,EnergyBins=energy,width=width,minPixel=minPixel,vmin=vmin,vmax=vmax,colorbar=False,cmap='turbo')
axIn.set_ylabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=34, color='w')
axIn.set_xlabel("")
axIn.tick_params(axis='both', which='major', labelsize=34, color='w')
axIn.set_ylim(0.26, 3.85)
axIn.set_xticks([0, 1], (r"$\Gamma$", r"$\left( \frac{1}{8} \frac{1}{8} 1 \right)$"), color='w');
axIn.tick_params(axis='y', colors='w');
axIn.get_figure().set_size_inches(4,5)
#plt.axvline(x=-0.02, color='w', linewidth=6.0, linestyle='--')

# Read in the simulated modes from SpinW. See https://mjolnir.readthedocs.io/en/latest/notebooks/OverviewofCAMEAData.html#The-MnF2-dispersion-theory, https://mjolnir.readthedocs.io/en/latest/notebooks/Fittingofsequential1Dcuts.html.
hklSplit = np.loadtxt('U:/Data/MATLAB/MnF2/hklFit10T.txt')
omegaSplit = np.loadtxt('U:/Data/MATLAB/MnF2/omegaFit10T.txt')
hklSplit = np.transpose(hklSplit)
hklSplit = hklSplit[0:223];
omegaSplit = omegaSplit[:, 0:223];
hklSplit1 = hklSplit[hklSplit[:,0]<Q2[0]]; # hkl's going up to Q2
omegaSplit1 = omegaSplit[:, hklSplit[:,0]<Q2[0]]
XPosition = np.concatenate([axIn.calculatePositionInv(hkl.reshape(1,-1)) for hkl in hklSplit1])
axIn.plot(XPosition, omegaSplit1[0], color='k', linewidth=2, linestyle='--')
axIn.plot(XPosition, omegaSplit1[1], color='k', linewidth=2, linestyle='--')
plt.savefig('U:/Data/Figures/MnF2/Manuscript/Fig4/HHLDataInset.png', transparent=True, bbox_inches='tight')

## Cut through Gamma

In [ ]:
dss=ds10T
Q1 = np.array([0,0,1])  #Q-position

width = 0.05 #integration width perpendicular to Q in 1/A
minPixel = 0.04 #step along Energy
EMin=0.5 #Minimum of energy
EMax=2.5 #Maximum of enerrgy
rlu = True
constantBins = True
scanvar='Energy'
fig, ax2 = plt.subplots(1, 1, figsize=(10,5))
data, bin = dss.cut1DE(E1=EMin,E2=EMax,q=Q1,width=width,minPixel=minPixel,rlu=rlu,constantBins=constantBins,ufit=False)
ax2.errorbar(data['Energy'],data['Int'],data['Int_err'], marker='o',linestyle='',markersize='10',capsize=3,elinewidth=3)

# Fit to two Gaussians and plot
#gmodel = Model(gaussian2)
#result = gmodel.fit(data['Int'], x=data['Energy'], bg=0.001, amp1=1, cen1=1.2, wid1=0.1, amp2=1, cen2=1.7, wid2=0.1)
#ax2.plot(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),
#gaussian2(np.linspace(np.array(data['Energy'])[0],np.array(data['Energy'])[-1],1000),result.params['bg'],
#result.params['amp1'],result.params['cen1'],result.params['wid1'],
#result.params['amp2'],result.params['cen2'],result.params['wid2']), linestyle='--', linewidth = 3, color='k')
ax2.set_title('')
ax2.set_xlabel(r'$\mathrm{\Delta}\mathit{E}$ (meV)', fontsize=18)
ax2.set_ylabel(r'$\it{I}$ (arb. units)', fontsize=18)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.get_figure().set_size_inches(10,6)

# Load in and overplot the simulated spectrum
ESim = np.loadtxt('U:/Data/MATLAB/MnF2/ESim10T.txt')
intSim = np.loadtxt('U:/Data/MATLAB/MnF2/intSim10T.txt')
ax2.plot(ESim, 4.3*intSim, linestyle='-', linewidth = 2, color='k')
ax2.set_xlim(0.5, 2.5);
plt.savefig('U:/Data/Figures/MnF2/Manuscript/Fig4/HHLCut.png')